# Make each level to each Curve CSV FILE

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_begin = []
num_row = []
f_1 = 'beginner_expert_processedData/beginner/beginner_'
f_3 = '.csv'
num_begin = 19
curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]

df_concat = pd.DataFrame()
# for curve_num in range(0,6):
for curve_num in range(0,1):
    for idx in range(1, num_begin+1):
        tmp_file = f_1+str(idx)+'_new2'+f_3
        df = pd.read_csv(tmp_file)
        df = df.dropna()
        
        tmp = df.astype(float)
        tmp['level'] =0
        
        tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]
        num_row.append(np.size(tmpcorner,0)) 
        
        df_begin.append(tmpcorner)
        df_concat = pd.concat([df_concat,df_begin[idx-1]])      
        
    df_concat.to_csv('cornerData/corner_'+str(curve_num+1)+'_begin'+'.csv')
    df_concat = pd.DataFrame()
    df_begin = []
    


In [3]:
df_exp = []
f_1 = 'beginner_expert_processedData/expert/expert_'
f_3 = '.csv'
num_exp = 19

df_concat = pd.DataFrame()

# for curve_num in range(0,6):
for curve_num in range(0,1):
    for idx in range(1, num_exp+1):
        tmp_file = f_1+str(idx)+'_new2'+f_3
        df = pd.read_csv(tmp_file)
        df = df.dropna()

        tmp = df.astype(float)
        tmp['level'] =1

        tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]
        num_row.append(np.size(tmpcorner,0)) 

        df_exp.append(tmpcorner)
        df_concat = pd.concat([df_concat,df_exp[idx-1]])
    
    df_concat.to_csv('cornerData/corner_'+str(curve_num+1)+'_expert'+'.csv')
    df_concat = pd.DataFrame()
    df_exp = []

In [4]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import random



left_column = [
#'Time',
    'Distance','GPS Latitude','GPS Longitude','Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
'Damper Velocity (Calc) RR','Corr Dist','Corr Dist (Unstretched)','Corr Speed','Brake Pos',
'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
'Self Align Torque RL','Self Align Torque RR','Session Time Left','Steering Angle','Suspension Travel FL','Suspension Travel FR',
'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
'CG Distance','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration','level']


#Hyper-parameters
num_epochs = 3
batches = 1
learning_rate = 0.001
input_size = len(left_column)-1 # left column except 'level'
output_size = 2 # Expert and Beginner
hidden_size = 128 # ?
num_layers = 2 
num_begin_train = 15
num_exp_train = 15
num_begin_test = num_begin - num_begin_train
num_exp_test = num_exp - num_exp_train

## Define GRU, Loss func and Optimizer
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.gru(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

gru = GRU(input_size, hidden_size, num_layers, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)  


## Data Processing
df_tmp_begin = pd.read_csv('cornerData/corner_1_begin.csv')
df_tmp_exp   = pd.read_csv('cornerData/corner_1_expert.csv')

df_curve1 = pd.concat([df_tmp_begin, df_tmp_exp], ignore_index=True) 
df_curve1= df_curve1.loc[:,left_column]
df_curve1_saved = df_curve1.loc[:,left_column] # data backup
df_curve1.to_csv('cornerData/corner_'+'_dfcurve1'+'.csv')

array_x = []
array_y = []
input_x = []
input_y = []
n_row = []

for i in range(0,num_begin + num_exp):
    x = df_curve1_saved.loc[0:num_row[i]-1]
    df_curve1_saved.drop(range(0,num_row[i]),inplace=True)
    df_curve1_saved.reset_index(drop=True, inplace=True)
    y = x.pop('level')
    array_x.append(x)
    array_y.append(y)

    
## Randomize sequence 
sequence = np.arange(num_begin + num_exp)
np.random.shuffle(sequence)
print(sequence)

for i in sequence:
    input_x.append(array_x[i])
    input_y.append(array_y[i])
    n_row.append(num_row[i])



## Train 
for epoch in range(num_epochs):
    for i in range(0,num_begin_train + num_exp_train):

        # array type (numpy)
        X = np.array(input_x[i])
        X = X.reshape(-1,n_row[i],input_size)
        Y = np.array(input_y[i])   
        
        # tensor type (pytorch)
        X = torch.from_numpy(X)
        X = X.float()
        Y = torch.tensor([Y[0]])

        output = gru(X)
        loss = criterion(output, Y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print (f'Loss: {loss.item():.4f}')

## Test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for i in range(num_begin_train + num_exp_train, num_begin + num_exp):
        
        # array type (numpy)
        X = np.array(input_x[i])
        X = X.reshape(-1,n_row[i],input_size)
        Y = np.array(input_y[i])   

        # tensor type (pytorch)
        X = torch.from_numpy(X)
        X = X.float()
        Y = torch.tensor([Y[0]])

        output = gru(X)
        _, predicted = torch.max(output.data, 1)
        n_samples += Y.size(0)
        n_correct += (predicted == Y).sum().item()        

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {num_begin_test + num_exp_test} test images: {acc} %')



[11 36 27  6 19  1 30 14  8  2 34 12  3 20 32 23  9 16  5  4  0 35 22 33
 18 28 13  7 25 29 37 10 31 26 21 24 17 15]
Loss: 0.6782
Loss: 1.1753
Loss: 0.9689
Loss: 0.5518
Loss: 0.8740
Loss: 0.4208
Loss: 0.8785
Loss: 0.8678
Loss: 0.9256
Loss: 0.4005
Loss: 0.4080
Loss: 0.4081
Loss: 0.4153
Loss: 0.8302
Loss: 1.0189
Loss: 1.1325
Loss: 0.6740
Loss: 0.2842
Loss: 0.3824
Loss: 0.2526
Loss: 0.2598
Loss: 0.5514
Loss: 1.9068
Loss: 0.3619
Loss: 0.3706
Loss: 1.0994
Loss: 1.0506
Loss: 0.4989
Loss: 0.2578
Loss: 1.6854
Loss: 0.2132
Loss: 0.2279
Loss: 0.4117
Loss: 0.5834
Loss: 0.4028
Loss: 0.4387
Loss: 0.2140
Loss: 1.2860
Loss: 0.8874
Loss: 0.3264
Loss: 0.2325
Loss: 0.3930
Loss: 0.5247
Loss: 0.3949
Loss: 1.1011
Loss: 0.8874
Loss: 0.6327
Loss: 0.2417
Loss: 0.2166
Loss: 0.1283
Loss: 0.1527
Loss: 0.5428
Loss: 1.8556
Loss: 0.4750
Loss: 0.2334
Loss: 0.5676
Loss: 0.1879
Loss: 0.2617
Loss: 0.1627
Loss: 1.2185
Loss: 0.1908
Loss: 0.1249
Loss: 0.2522
Loss: 0.6559
Loss: 0.2743
Loss: 0.4733
Loss: 0.1743
Loss: 1.0967